In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# !pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
label2id = {'negative':0,'positive':1}

train = pd.read_csv(r'train.csv', encoding='utf_8_sig')
train = train.replace({"label": label2id})
train.fillna('', inplace=True)
test = pd.read_csv(r'test.csv', encoding='utf_8_sig')
test = test.replace({"label": label2id})
test.fillna('', inplace=True)

/tmp/ipykernel_2496/2093326092.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train = train.replace({"label": label2id})
/tmp/ipykernel_2496/2093326092.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test = test.replace({"label": label2id})


In [4]:
train

,label,text,diff
0,1,Check types to avoid invalid reads/writes.,diff --git a/src/file.c b/src/file.c\nindex 4d...
1,1,[CVE-2012-2098] Integrate fallback sort into t...,diff --git a/src/main/java/org/apache/commons/...
2,0,Merge branch '1.8.x',diff --git a/src/ClientCommand.cpp b/src/Clien...
3,0,[JBPM-9474] Produce events to Kafka by Signal ...,diff --git a/jbpm-bpmn2/src/main/java/org/jbpm...
4,1,Resolving Issue #918\n\nMinor XSS and create g...,diff --git a/docs/CHANGELOG b/docs/CHANGELOG\n...
5,1,CVE-2017-12895/ICMP: Check the availability of...,diff --git a/print-icmp.c b/print-icmp.c\ninde...
6,0,9.0.4,diff --git a/package.json b/package.json\ninde...
7,0,"Needs to allow this syscall as well, caught on...",diff --git a/linux_priv.c b/linux_priv.c\ninde...
8,1,SPOOLSS: Try to avoid an infinite loop.\n\nUse...,diff --git a/epan/dissectors/packet-dcerpc-spo...
9,1,Update NewslettersController.php\n\nTest if an...,diff --git a/www/guis/users/application/contro...


In [5]:
len(train)

40

In [6]:
train['label'].value_counts()

label
0    25
1    15
Name: count, dtype: int64

In [7]:
test['label'].value_counts()

label
0    6322
1    3750
Name: count, dtype: int64

In [8]:
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer
# model_id = "../../sentence-transformers/paraphrase-mpnet-base-v2"
model = SentenceTransformer('../../sentence-transformers/paraphrase-mpnet-base-v2')

In [9]:
train['embeddings'] = train['text'].apply(lambda x: model.encode(str(x)))
test['embeddings'] = test['text'].apply(lambda x: model.encode(str(x)))

In [10]:
test

,label,text,diff,embeddings
0,1,Escapes now HTML (JsonBuilder).,diff --git a/src/main/java/org/projectforge/we...,"[-0.041513152, 0.24733369, -0.07403697, 0.0535..."
1,1,Fix CSV injection issue if server responds wit...,diff --git a/program/plugins/nikto_outdated.pl...,"[-0.009063438, -0.27374047, -0.004000899, 0.03..."
2,0,Merge PR #4713: BUILD(installer): Include g15-...,diff --git a/installer/ClientInstaller.cs b/in...,"[-0.18374333, 0.22310452, -0.004842472, -0.082..."
3,0,Merge pull request #153 from jglick/dependabot...,diff --git a/.github/CODEOWNERS b/.github/CODE...,"[-0.11300747, 0.20437212, -0.013591562, 0.0282..."
4,1,https://github.com/ImageMagick/ImageMagick/iss...,diff --git a/coders/txt.c b/coders/txt.c\ninde...,"[-0.1070229, 0.2002589, 0.04798811, 0.00870072..."
...,...,...,...,...
10067,1,https://github.com/ImageMagick/ImageMagick/iss...,diff --git a/MagickWand/mogrify.c b/MagickWand...,"[-0.090522535, 0.075812824, 0.045917258, 0.038..."
10068,1,https://github.com/ImageMagick/ImageMagick/iss...,diff --git a/coders/mat.c b/coders/mat.c\ninde...,"[-0.0847711, 0.12157971, 0.035356823, 0.024104..."
10069,0,[maven-release-plugin] prepare release promote...,diff --git a/pom.xml b/pom.xml\nindex 92353944...,"[-0.02828614, -0.03947256, -0.05639892, -0.140..."
10070,1,Taxonomy: Correct the formatting of HTML entit...,diff --git a/wp-admin/js/tags-box.js b/wp-admi...,"[-0.08080342, 0.09955617, -0.04532254, 0.09845..."


In [11]:
X_train = train['embeddings'].to_list()
X_test = test['embeddings'].to_list()

y_train = train['label'].to_list()
y_test = test['label'].to_list()

In [12]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [13]:
from sklearn import metrics
predicted = LR.predict(X_test)
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.f1_score(y_test, predicted,average='weighted'))
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))

Logistic Regression Precision: 0.742779743473779
Logistic Regression Recall: 0.7477164416203336
Logistic Regression Recall: 0.7414516343507467
Logistic Regression Accuracy: 0.7477164416203336
